# This notebook is for generating R2RML File from CSV file

In [2]:
# !pip install flask
import os
import psycopg2
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout, Button, Box, interact, interactive, fixed, interact_manual, Checkbox
from PIL import Image
import pandas as pd
from collections import Counter

In [135]:
df=pd.read_csv("BLK_earlyLife_agingTrend.csv", sep=',')
print('The number of variables is ', len(df.columns))

The number of variables is  137


/Users/changsun/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [136]:
variables = list(df.columns)
varType = list(df.dtypes)

In [156]:
tableName = 'EarlyLifeAgeTrend'
tableStructure = 'CREATE TABLE %s(' %tableName
for i in range(0, len(varType)):
    if varType[i] == 'int64':
        tempType = 'int'
    else:
        tempType = 'real'

    if str(variables[i]) == 'PRIMARY':
        temp = 'PRIMARYSCHOOL'+' '+tempType+','
    else:
        temp = str(variables[i])+' '+tempType+','
    tableStructure = tableStructure + temp 
tableStructure = tableStructure[:-1] + ')'

In [157]:
con = psycopg2.connect("host='localhost' dbname='changsun' user='postgres' password='postgres'") # host='CSdatabase', dbname='postgres'
print("Server Connected!\n")
cur = con.cursor()
cur.execute("DROP TABLE EarlyLifeAgeTrend")
cur.execute(tableStructure)
print("Darling, Table is created for you successfully.")

Server Connected!

Darling, Table is created for you successfully.


In [158]:
# Import CSV Data
dataFile = "BLK_earlyLife_agingTrend.csv" 
copy_sql = """
           COPY %s FROM stdin WITH (FORMAT CSV, HEADER true, NULL '', DELIMITER ',')           
           """ %tableName
with open(dataFile, 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
con.commit()
# cur.close()

In [146]:
# # Import CSV Data
# dataFile = "BLANK_earlyLife_agingTrend.csv" # 'NLD/REGION_DEMOGR_Mortality.csv'
# with open(dataFile, 'r') as f:
#     # Notice that we don't need the `csv` module.
#     next(f)  # Skip the header row.
#     cur.copy_from(f, tableName, sep=',') #copy_from doesn't support delimitor in quota    
# con.commit()

In [143]:
df=pd.read_csv("BLK_earlyLife_agingTrend.csv", delimiter=',')
col = df.columns

In [154]:
style = {'description_width': 'initial'}
RDFTerm = ['child','class', 'column', 'constant', 'datatype', 'graph', 'graphMap', \
            'inverseExpression', 'joinCondition', 'language', 'parent', 'parentTriplesMap', \
            'sqlVersion', 'termtype', 'template']
tableClassTerm = ['sqlQuery', 'tableName']
def inputInfo():
    # set prefix
    uniLayout = Layout(width='70%', height='30px')
    
    common = "@prefix rr: <http://www.w3.org/ns/r2rml#>.\n@prefix ex: <http://earlylifeagetrend.com/base#>."
    prefixText = widgets.Textarea(value=common, description='Input Prefix', style=style,\
                                  layout=Layout(width='50%', height='100px'))

    # set map name
    common = '<#TriplesMap1>'
    mapName = widgets.Text(value=common, description='Give a name to this Map: ', style=style,\
                                  layout=uniLayout)

    # set name of logical table:
    tableClass = widgets.Dropdown(options=tableClassTerm, layout=uniLayout, \
                                   description='Choose Table Type: ', style=style)
    tableName = widgets.Textarea(value='SELECT * FROM EarlyLifeAgeTrend;', description='Table Name or SQL code:', \
                             layout=Layout(width='70%', height='70px'), style=style)
    
    # R2RML Properties
    subRDFTermSet = widgets.Dropdown(options=RDFTerm, layout=uniLayout, \
                                   description='Choose a RDF Term: ', style=style)
    
    # set subjectMap
    subWeb = widgets.Text(value='http://data.earlylifeagetrend.com/patient/', layout=uniLayout, \
                          description='Input your content:', style=style)
    subClass = widgets.Dropdown(options=list(col), \
                                description='Choose subject from logical table: ', \
                                layout=uniLayout, \
                                style=style)
    subName = widgets.Text(value='Patient', description='Give a name to this suject: ', \
                           layout=uniLayout, style=style)
    
    return prefixText, mapName, tableClass, tableName, subRDFTermSet, subWeb, subClass, subName

def inputObj(flagB):
    uniLayout = Layout(width='70%', height='30px')
    # set predicateObjectMap:
    objRDFTermSet = widgets.Dropdown(options=RDFTerm, layout=uniLayout, \
                                   description='Choose a RDF Term: ', style=style)
    objWeb = widgets.Text(value='PregnancyTimes', layout=uniLayout, \
                          description='Give a name to this predicate: ', style=style)
    objClass = widgets.Dropdown(options=list(col), layout=uniLayout, \
                                description='Choose object from logical table:', style=style)
    objNum = widgets.Text(value='1', layout=uniLayout, description='Number of Objects(Only integer): ', \
                          style=style)
    if flagB:
        return objRDFTermSet, objWeb, objClass, objNum
    else:
        return objRDFTermSet, objWeb, objClass     

In [159]:
prefixText, mapName, tableClass, tableName, subRDFTermSet, subWeb, subClass, subName = inputInfo()
objRDFTermSet, objWeb, objClass, objNum = inputObj(True)
para = [prefixText, mapName, tableName, subRDFTermSet, objRDFTermSet, subWeb, \
subClass, subName, objWeb, objClass]

group1 = [mapName, tableClass, tableName]
group2 = [subClass, subName, subRDFTermSet, subWeb]
group3 = [objWeb, objRDFTermSet, objClass, objNum]

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='70%')
box_layout2 = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='70%')
box1 = Box(children=group1, layout=box_layout)
box2 = Box(children=group2, layout=box_layout)
box3 = Box(children=group3, layout=box_layout)

steps = [prefixText, box1, box2, box3]
tab = widgets.Tab(children=steps, style=style)
tab.set_title(0, 'Step0--Prefix')
tab.set_title(1, 'Step1--Name')
tab.set_title(2, 'Step2--Subject')
tab.set_title(3, 'Step3--Pre&Obj')
    
action = widgets.Dropdown(options=['Add More Objects', 'Check Info', 'Generate File'],\
                          description='Action: ', style=style)
button = widgets.Button(description="RUN", button_style='danger')
boxACTION = Box(children=[action,button], layout=box_layout2)
display(tab, boxACTION)

add_objRDFTermSet=[None]*1000
add_objWeb=[None]*1000
add_objClass=[None]*1000
saveButton=[None]*1000

def actionFunction(x):
    
    if action.value == 'Add More Objects': 
        
        for i in range(0, int(objNum.value)-1):
            add_objRDFTermSet[i], add_objWeb[i], add_objClass[i] = inputObj(False) 
            saveButton[i] = widgets.Checkbox(value=False, \
                            description='I am sure to save this object', style=style)
            group4 = [add_objWeb[i], add_objRDFTermSet[i], add_objClass[i], saveButton[i]]
            add_box_layout = Layout(display='flex',
                            flex_flow='column',
                            border='solid',
                            align_items='stretch',
                            width='70%')
            box4 = Box(children=group4, layout=add_box_layout)
            display(box4)
        
    else:
        flag = True
        print('******** Check the information you inputed ********')
        for p in para:
            # check if all info were filled
            if not p.value:
                flag = False
                print('Please input all information!')

        if flag:
            add_preObjMap = ''
            tableView = '<#TableView> rr:sqlQuery """ %s """.'%tableName.value
            logTable = 'rr:logicalTable <#TableView>;' 
            subMap = """rr:subjectMap [\n\t\trr:%s "%s{%s}";\n\t\trr:class ex:%s;\n\t];"""\
            %(subRDFTermSet.value, subWeb.value, subClass.value, subName.value)
            preObjMap = """rr:predicateObjectMap [\n\t\trr:predicate ex:%s;\n\t\trr:objectMap [ rr:%s "%s" ];\n\t];"""%(objWeb.value, objRDFTermSet.value, objClass.value)
            for i in range(0, int(objNum.value)-1):
                if saveButton[i].value == True:
                    preObjMap = preObjMap+"""\n\trr:predicateObjectMap [\n\t\trr:predicate ex:%s;\n\t\trr:objectMap [ rr:%s "%s" ];\n\t];"""%(add_objWeb[i].value, add_objRDFTermSet[i].value, add_objClass[i].value)
                    
            fullText = prefixText.value+'\n\n'+tableView+'\n\n'+mapName.value+'\n\t'+logTable+'\n\t'+subMap\
                        +'\n\t'+preObjMap[:-1]+'.'
            if action.value=='Check Info':    
                print(fullText)
            if action.value=='Generate File':
                write_text = open("R2RML.ttl", "w")
                write_text.write(fullText)
                write_text.close()
                print('Your file: ', os.getcwd())
            
# interact_manual(actionFunction, x=action);
# doneButton.on_click(addObject)
button.on_click(actionFunction)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

******** Check the information you inputed ********
@prefix rr: <http://www.w3.org/ns/r2rml#>.
@prefix ex: <http://earlylifeagetrend.com/base#>.

<#TableView> rr:sqlQuery """ SELECT * FROM EarlyLifeAgeTrend; """.

<#TriplesMap1>
	rr:logicalTable <#TableView>;
	rr:subjectMap [
		rr:template "http://data.earlylifeagetrend.com/patient/{COUNTRY}";
		rr:class ex:country_study;
	];
	rr:predicateObjectMap [
		rr:predicate ex:has_enrolled;
		rr:objectMap [ rr:template "ID" ];
	];
	rr:predicateObjectMap [
		rr:predicate ex:has_enrolled;
		rr:objectMap [ rr:column "ID" ];
	].


In [ ]:

# <http://data.VWDataRDF.com/MS/participant/Felic_Pryer>
#         a                            ex:participant ;
#         ex:diabetes_status_is        "4" ;
#         ex:has_diabetes_for          "0" ;
#         ex:has_measured              <http://data.VWDataRDF.com/MS/plasma_ages> , <http://data.VWDataRDF.com/MS/beta_Cell_Function> , <http://data.VWDataRDF.com/MS/diabetes_status> , <http://data.VWDataRDF.com/MS/neuropathy> , <http://data.VWDataRDF.com/MS/paid> , <http://data.VWDataRDF.com/MS/retinopathy>  , <http://data.VWDataRDF.com/MS/nephropathy> , <http://data.VWDataRDF.com/MS/newly_type2> , <http://data.VWDataRDF.com/MS/cardiovascular_health> , <http://data.VWDataRDF.com/MS/glucose_cpeptide_insulin> , <http://data.VWDataRDF.com/MS/diabetes_duration> ;
#         ex:has_value_of_betaCellFunction
#                 "0.69999999" ;
#         ex:has_value_of_cardiovascular_health
#                 "4" ;
#         ex:has_value_of_insulin      "13.9" ;
#         ex:has_value_of_nephropathy  "3.5"^^<http://www.w3.org/2001/XMLSchema#double> ;
#         ex:has_value_of_neuropathy   "6.5599999"^^<http://www.w3.org/2001/XMLSchema#double> ;
#         ex:has_value_of_newlytype2   "1.11"^^<http://www.w3.org/2001/XMLSchema#double> ;
#         ex:has_value_of_paid         "2.8800001" ;
#         ex:has_value_of_plasma       "6.73"^^<http://www.w3.org/2001/XMLSchema#double> ;
#         ex:has_value_of_retinopathy  "7.8299999"^^<http://www.w3.org/2001/XMLSchema#double> ;
#         ex:was_born_on               "18/01/1977" .

# <http://data.VWDataRDF.com/MS/participant/Ramsey_Precious>
#         a                            ex:participant ;
#         ex:diabetes_status_is        "1" ;
#         ex:has_diabetes_for          "39" ;
#         ex:has_measured              <http://data.VWDataRDF.com/MS/nephropathy> , <http://data.VWDataRDF.com/MS/plasma_ages> , <http://data.VWDataRDF.com/MS/newly_type2> , <http://data.VWDataRDF.com/MS/paid> , <http://data.VWDataRDF.com/MS/diabetes_status>  , <http://data.VWDataRDF.com/MS/diabetes_duration> , <http://data.VWDataRDF.com/MS/beta_Cell_Function> , <http://data.VWDataRDF.com/MS/glucose_cpeptide_insulin> , <http://data.VWDataRDF.com/MS/neuropathy> , <http://data.VWDataRDF.com/MS/retinopathy> , <http://data.VWDataRDF.com/MS/cardiovascular_health> ;
#         ex:has_value_of_betaCellFunction
#                 "10.48" ;
#         ex:has_value_of_cardiovascular_health
#                 "6" ;
#         ex:has_value_of_insulin      "6.9299998" ;
#         ex:has_value_of_nephropathy  "0.99000001"^^<http://www.w3.org/2001/XMLSchema#double> ;
#         ex:has_value_of_neuropathy   "8.5799999"^^<http://www.w3.org/2001/XMLSchema#double> ;
#         ex:has_value_of_newlytype2   "2.75"^^<http://www.w3.org/2001/XMLSchema#double> ;
#         ex:has_value_of_paid         "23.27" ;
#         ex:has_value_of_plasma       "4.9099998"^^<http://www.w3.org/2001/XMLSchema#double> ;
#         ex:has_value_of_retinopathy  "0.97000003"^^<http://www.w3.org/2001/XMLSchema#double> ;
#         ex:was_born_on               "03/07/1961" .
